In [1]:
import spacy
import string
import re
import pandas as pd

In [2]:
# 한국어 모델 로드
nlp = spacy.load("ko_core_news_sm")

def noise_score_spacy(text):
    doc = nlp(text)  # 입력된 텍스트 토큰화
    word_count = len(doc) # 토큰 수 count
    
    # 노이즈 관련 점수 계산
    special_char_count = sum(1 for char in text if char in string.punctuation) # 특수문자의 개수 세기: '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'
    non_korean_char_count = len(re.findall(r'[a-zA-Z0-9]', text))  # 한글이 아닌 문자(알파벳, 숫자) 개수 세기
    short_word_count = sum(1 for token in doc if len(token.text) == 1) # 길이가 1인 단어 수 세기 (노이즈 가능성 높음)
    separated_hangul_count = len(re.findall(r'[ㄱ-ㅎㅏ-ㅣ]', text)) # 한글 자모 분리 정도 감지 (ㅋ, ㄱ, ㅏ 등)
    irregular_case_count = sum(1 for i in range(1, len(text)) if text[i-1].islower() and text[i].isupper()) # 불규칙한 대소문자 패턴 (소문자 다음 대문자)
    
    # 품사 태그 다양성 (낮을수록 텍스트 단조롭고 노이즈 가능성 높을 것)
    pos_counts = {}
    for token in doc:
        pos_counts[token.pos_] = pos_counts.get(token.pos_, 0) + 1
    pos_diversity = len(pos_counts)
    
    # 노이즈 점수 계산
    noise_score = (
        (special_char_count * 2) +
        (non_korean_char_count * 1.5) +
        (short_word_count * 1.5) +
        (separated_hangul_count * 3) +
        (irregular_case_count * 2)
    ) / max(1, word_count)  # 단어 수로 나누어 정규화
    
    # 품사 다양성에 따른 보정 (다양성이 낮을수록 점수 증가)
    diversity_factor = max(1, 5 - pos_diversity) / 5
    noise_score *= (1 + diversity_factor)
    
    # 최종 점수 범위 조정 (0-100 사이)
    final_score = min(100, max(0, noise_score * 20))
    
    return final_score

In [3]:
train = pd.read_csv("../../data/train.csv")
train['noise_score'] = train['text'].apply(noise_score_spacy)

In [4]:
# noise 점수가 30 이상 70 미만인 것들만 재작성. / 70 이상인건 삭제
data_denoise = train[(train.noise_score >= 30) & (train.noise_score < 70)].reset_index(drop=True)

In [5]:
data_denoise

,ID,text,target,noise_score
0,ynat-v1_train_00000,정i :파1 미사z KT( 이용기간 2e 단] Q분종U2보,4,58.909091
1,ynat-v1_train_00002,"m 김정) 자주통일 새,?r열1나가야1보",2,61.714286
2,ynat-v1_train_00004,pI美대선I앞두고 R2fr단 발] $비해 감시 강화,6,63.000000
3,ynat-v1_train_00006,프로야구~롯TKIAs광주 경기 y천취소,1,60.000000
4,ynat-v1_train_00007,아가메즈 33득점 우리카드 KB손해보험 완파…3위 굳...,4,44.000000
...,...,...,...,...
809,ynat-v1_train_02784,"고려·vq 5|째 결승 격돌la~일,%3대o농구 챔피Tw",1,60.000000
810,ynat-v1_train_02788,남원소식 춘>X학%단+장Rn 모집,0,64.800000
811,ynat-v1_train_02789,"이총리,세4. H직 완fl지 못해E할 일 꽤;남아",2,48.000000
812,ynat-v1_train_02790,프로농구 개막 ③ 헤인즈의 1만점·양동근의 900스틸…...,3,39.600000


## DeepL 번역기 - RTT
- 중간 언어: 일본어

In [ ]:
# !pip install deepl

In [ ]:
import deepl
import time

auth_key = "(키 입력)"
translator = deepl.Translator(auth_key)

In [19]:
data_denoise['trans_japan'] = ""
data_denoise['trans_final'] = ""

In [20]:
def translate_with_retry(translator, text, source_lang, target_lang, max_retries=5):
    for attempt in range(max_retries):
        try:
            return translator.translate_text(text, source_lang=source_lang, target_lang=target_lang).text
        except deepl.DeepLException as e:
            if attempt < max_retries - 1:
                print(f"Translation error occurred: {e}. Retrying in 3 seconds...")
                time.sleep(3)
            else:
                print(f"Max retries reached. Translation failed for text: {text}")
                return None

for idx, row in data_denoise.iterrows():
    text = row['text']
    
    trans_japan = translate_with_retry(translator, text, source_lang="KO", target_lang="JA")
    if trans_japan is not None:
        data_denoise.loc[idx, 'trans_japan'] = trans_japan
        
        trans_final = translate_with_retry(translator, trans_japan, source_lang="JA", target_lang="KO")
        if trans_final is not None:
            data_denoise.loc[idx, 'trans_final'] = trans_final
            print(idx, text, "->", trans_final)
        else:
            print(f"Failed to translate from Japanese to Korean for index {idx}")
    else:
        print(f"Failed to translate from Korean to Japanese for index {idx}")

    if idx == 5: 
        break

0 정i :파1 미사z KT( 이용기간 2e 단] Q분종U2보 -> 鄭i:파1ミサz KT(利用期間2e単])Q分種U2保
1 m 김정) 자주통일 새,?r열1나가야1보 -> m 김정은) 자주통일의 새,?
2 pI美대선I앞두고 R2fr단 발] $비해 감시 강화 -> pI 미국 대통령 선거를 앞두고 R2fr 끝발] $ 비수역 감시 강화
3 프로야구~롯TKIAs광주 경기 y천취소 -> 프로야구~롯데TKIAs 광주 경기 중단
4 아가메즈 33득점 우리카드 KB손해보험 완파…3위 굳... -> 아가메즈 33득점 우리카드 KB손해보험에 완승.... 3위 확정...
5 oi 매력 R모h츠a열#w3약 >l·주가 고Q/진 -> oi 매력 R 모차르 열 #w3 약 >l- 주식 고Q/진


In [ ]:
data_denoise.to_csv("../../datasets/v4.0.1/bt_japan.csv", index=False)

,ID,text,target,noise_score,trans_japan,trans_final
0,ynat-v1_train_00000,정i :파1 미사z KT( 이용기간 2e 단] Q분종U2보,4,58.909091,鄭i:파1ミサz KT( 利用期間2e単])Q分種U2保,鄭i:파1ミサz KT(利用期間2e単])Q分種U2保
1,ynat-v1_train_00002,"m 김정) 자주통일 새,?r열1나가야1보",2,61.714286,m 金正恩) 自主統一の鳥、?,"m 김정은) 자주통일의 새,?"
2,ynat-v1_train_00004,pI美대선I앞두고 R2fr단 발] $비해 감시 강화,6,63.000000,pI美大統領選Iを前にR2fr端足] $非海域監視強化,pI 미국 대통령 선거를 앞두고 R2fr 끝발] $ 비수역 감시 강화
3,ynat-v1_train_00006,프로야구~롯TKIAs광주 경기 y천취소,1,60.000000,,
4,ynat-v1_train_00007,아가메즈 33득점 우리카드 KB손해보험 완파…3위 굳...,4,44.000000,,
